In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from flask import Flask, render_template, flash, request
from datetime import datetime
from datetime import timedelta

app = Flask(__name__)


In [2]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-03-28T10:13:09.412Z,64.441700,-149.50200,13.8000,1.00,ml,NaN,NaN,NaN,0.21,...,2024-03-28T10:15:40.869Z,"18 km NW of Anderson, Alaska",earthquake,NaN,0.800000,NaN,NaN,automatic,ak,ak
1,2024-03-28T10:11:17.960Z,38.818333,-122.77433,2.2200,0.78,md,7.0,226.0,0.007093,0.11,...,2024-03-28T10:12:52.679Z,"4 km W of Cobb, CA",earthquake,1.8,1.940000,0.19,9.0,automatic,nc,nc
2,2024-03-28T09:56:10.780Z,34.223833,-117.42800,13.7100,0.98,ml,39.0,34.0,0.046220,0.14,...,2024-03-28T10:02:39.701Z,"2 km W of Devore, CA",earthquake,0.2,0.290000,0.30,30.0,automatic,ci,ci
3,2024-03-28T09:42:36.069Z,31.602000,-103.97200,4.6004,1.60,ml,16.0,64.0,0.000000,0.20,...,2024-03-28T10:02:11.012Z,"36 km NNW of Toyah, Texas",earthquake,0.0,1.393366,0.20,13.0,automatic,tx,tx
4,2024-03-28T09:24:41.458Z,58.103900,-155.49870,11.0000,1.00,ml,NaN,NaN,NaN,0.67,...,2024-03-28T09:26:19.117Z,"85 km NW of Karluk, Alaska",earthquake,NaN,0.200000,NaN,NaN,automatic,ak,ak


In [3]:
df = df.sort_values('time', ascending=True)
df['date'] = df['time'].str[0:10]

In [4]:
df = df[['date', 'latitude', 'longitude', 'depth', 'mag', 'place']]
temp_df = df['place'].str.split(', ', expand=True) 
df['place'] = temp_df[1]
df = df[['date', 'latitude', 'longitude', 'depth', 'mag', 'place']]

In [5]:
df_coords = df[['place', 'latitude', 'longitude']]
df_coords = df_coords.groupby(['place'], as_index=False).mean()
df_coords = df_coords[['place', 'latitude', 'longitude']]

In [6]:
df = df[['date', 'depth', 'mag', 'place']]
df = pd.merge(left=df, right=df_coords, how='inner', on=['place'])

In [7]:
eq_data = []
df_live = []

In [8]:
days_out_to_predict=25
for symbol in list(set(df['place'])):
        temp_df = df[df['place'] == symbol].copy()
        temp_df['depth_avg_22'] = temp_df['depth'].rolling(window=22,center=False).mean() 
        temp_df['depth_avg_15'] = temp_df['depth'].rolling(window=15,center=False).mean()
        temp_df['depth_avg_7'] = temp_df['depth'].rolling(window=7,center=False).mean()
        temp_df['mag_avg_22'] = temp_df['mag'].rolling(window=22,center=False).mean() 
        temp_df['mag_avg_15'] = temp_df['mag'].rolling(window=15,center=False).mean()
        temp_df['mag_avg_7'] = temp_df['mag'].rolling(window=7,center=False).mean()
        temp_df.loc[:, 'mag_outcome'] = temp_df.loc[:, 'mag_avg_7'].shift(days_out_to_predict * -1)

        df_live.append(temp_df.tail(days_out_to_predict))

        eq_data.append(temp_df)

In [9]:
df = pd.concat(eq_data)
df = df[np.isfinite(df['depth_avg_22'])]
df = df[np.isfinite(df['mag_avg_22'])]
df = df[np.isfinite(df['mag_outcome'])]
df['mag_outcome'] = np.where(df['mag_outcome'] > 2.5, 1,0)

In [10]:
df = df[['date','latitude','longitude','depth_avg_22','depth_avg_15','depth_avg_7','mag_avg_22', 'mag_avg_15','mag_avg_7','mag_outcome']]

In [11]:
df_live = pd.concat(df_live)
df_live = df_live[np.isfinite(df_live['mag_avg_22'])]

In [12]:
df_live.head()

,date,depth,mag,place,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
8399,2024-03-20,0.0,1.7,California,40.589008,-120.497151,6.657727,4.458000,2.200000,1.201818,1.356000,1.542857,NaN
8400,2024-03-20,9.3,0.5,California,40.589008,-120.497151,6.553182,4.578000,2.442857,1.206364,1.396000,1.400000,NaN
8401,2024-03-20,15.1,0.7,California,40.589008,-120.497151,6.671364,5.584667,3.485714,1.174545,1.309333,1.242857,NaN
8402,2024-03-20,1.5,1.2,California,40.589008,-120.497151,6.335000,4.871333,3.700000,1.165455,1.362667,1.171429,NaN
8403,2024-03-20,7.9,0.6,California,40.589008,-120.497151,5.857727,4.878000,4.828571,1.156364,1.336000,1.171429,NaN


In [13]:
from sklearn.model_selection import train_test_split
features = [f for f in list(df) if f not in ['date', 'mag_outcome', 'latitude','longitude']]

In [14]:
df[features].head()

,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7
8313,7.213636,8.146667,10.0,2.290909,2.880000,3.514286
8314,7.268182,8.320000,10.0,2.418182,3.006667,3.385714
8315,7.095455,8.986667,10.0,2.595455,3.166667,3.528571
8316,7.236364,9.033333,10.0,2.795455,3.400000,3.714286
8317,7.640909,9.366667,10.0,2.804545,3.446667,3.657143


In [15]:
df['mag_outcome'].head()

8313    0
8314    0
8315    0
8316    0
8317    0
Name: mag_outcome, dtype: int32

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df[features],df['mag_outcome'], test_size=0.3, random_state=42)

In [17]:
lstm_units=50 
epochs=50
batch_size=72
    

In [18]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [20]:
model = Sequential()
model.add(LSTM(lstm_units, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(X_train_lstm, y_train, epochs=100, batch_size=batch_size, validation_data=(X_test_lstm, y_test), verbose=2, shuffle=False)


Epoch 1/100


82/82 - 5s - loss: 0.5816 - accuracy: 0.9250 - val_loss: 0.4388 - val_accuracy: 0.9798 - 5s/epoch - 63ms/step
Epoch 2/100
82/82 - 1s - loss: 0.3209 - accuracy: 0.9768 - val_loss: 0.2299 - val_accuracy: 0.9798 - 501ms/epoch - 6ms/step
Epoch 3/100
82/82 - 1s - loss: 0.2071 - accuracy: 0.9768 - val_loss: 0.1773 - val_accuracy: 0.9798 - 557ms/epoch - 7ms/step
Epoch 4/100
82/82 - 1s - loss: 0.1769 - accuracy: 0.9768 - val_loss: 0.1556 - val_accuracy: 0.9798 - 517ms/epoch - 6ms/step
Epoch 5/100
82/82 - 0s - loss: 0.1589 - accuracy: 0.9768 - val_loss: 0.1389 - val_accuracy: 0.9798 - 470ms/epoch - 6ms/step
Epoch 6/100
82/82 - 1s - loss: 0.1433 - accuracy: 0.9768 - val_loss: 0.1243 - val_accuracy: 0.9798 - 500ms/epoch - 6ms/step
Epoch 7/100
82/82 - 0s - loss: 0.1295 - accuracy: 0.9768 - val_loss: 0.1116 - val_accuracy: 0.9798 - 486ms/epoch - 6ms/step
Epoch 8/100
82/82 - 0s - loss: 0.1177 - accuracy: 0.9768 - val_loss: 0.1009 - val_accuracy: 0.9798 - 485ms/epoch - 6ms/step
Epoch 9/

82/82 - 0s - loss: 0.0336 - accuracy: 0.9874 - val_loss: 0.0235 - val_accuracy: 0.9905 - 464ms/epoch - 6ms/step
Epoch 64/100
82/82 - 0s - loss: 0.0336 - accuracy: 0.9874 - val_loss: 0.0235 - val_accuracy: 0.9905 - 454ms/epoch - 6ms/step
Epoch 65/100
82/82 - 0s - loss: 0.0336 - accuracy: 0.9874 - val_loss: 0.0235 - val_accuracy: 0.9905 - 448ms/epoch - 5ms/step
Epoch 66/100
82/82 - 0s - loss: 0.0336 - accuracy: 0.9874 - val_loss: 0.0235 - val_accuracy: 0.9905 - 455ms/epoch - 6ms/step
Epoch 67/100
82/82 - 0s - loss: 0.0336 - accuracy: 0.9876 - val_loss: 0.0234 - val_accuracy: 0.9905 - 469ms/epoch - 6ms/step
Epoch 68/100
82/82 - 0s - loss: 0.0335 - accuracy: 0.9876 - val_loss: 0.0234 - val_accuracy: 0.9905 - 463ms/epoch - 6ms/step
Epoch 69/100
82/82 - 0s - loss: 0.0335 - accuracy: 0.9876 - val_loss: 0.0234 - val_accuracy: 0.9905 - 471ms/epoch - 6ms/step
Epoch 70/100
82/82 - 0s - loss: 0.0335 - accuracy: 0.9876 - val_loss: 0.0234 - val_accuracy: 0.9905 - 469ms/epoch - 6ms/step
Epoch 71/100


In [22]:
df_live_scaled = scaler.transform(df_live[features])
df_live_lstm = df_live_scaled.reshape((df_live_scaled.shape[0], 1, df_live_scaled.shape[1]))
preds = model.predict(df_live_lstm)

15/15 [==============================] - 1s 4ms/step


In [23]:
df_live_result = df_live[['date', 'place', 'latitude', 'longitude']]
df_live_result['preds'] = preds
df_live_result = df_live_result.groupby(['date', 'place'], as_index=False).mean()


C:\Users\Admin\AppData\Local\Temp\ipykernel_14460\3804358955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_live_result['preds'] = preds


In [24]:
df_live_result['date'] = pd.to_datetime(df_live_result['date'], format='%Y-%m-%d') + pd.to_timedelta(days_out_to_predict, unit='d')
pred=df_live_result

In [25]:
def get_earth_quake_estimates(desired_date, df_live):
    from datetime import datetime
    live_set_tmp = df_live[df_live['date'] == desired_date]
    LatLngString = ''
    place1=''
    if (len(live_set_tmp) > 0):
        for lat, lon, pred,place in zip(live_set_tmp['latitude'], live_set_tmp['longitude'], live_set_tmp['preds'],live_set_tmp['place']): 
            if (pred > 0.5):
                LatLngString += "new google.maps.LatLng(" + str(lat) + "," + str(lon) + "),"
                place = f"{place}-{pred*100:.2f}"
                place1 += " " +place
    return (LatLngString),place1

In [26]:
def get_earth_quake_estimates1(desired_place, df_live):
    from datetime import datetime
    live_set_tmp = df_live[df_live['place'] == desired_place]
    LatLngString = ''
    place1=''
    if (len(live_set_tmp) > 0):
        for lat, lon, pred, date in zip(live_set_tmp['latitude'], live_set_tmp['longitude'], live_set_tmp['preds'], live_set_tmp['date']): 
            if (pred > 0.5):
                LatLngString += "new google.maps.LatLng(" + str(lat) + "," + str(lon) + "),"
                date_str = date.strftime('%Y-%m-%d')  # Format date as 'YYYY-MM-DD'
                place = f"{date_str}-\"{pred*100:.2f}\""
                place1 += " " + place

    return LatLngString, place1




In [27]:
@app.route("/", methods=['POST', 'GET'])
def build_page():
        if request.method == 'POST':
            date=request.form["date"]
            earthquake_horizon,place= get_earth_quake_estimates(date, pred)
            place=place.split()
            #print(place)
            return render_template('index1.html',
                earthquake_horizon = earthquake_horizon,
                place=place,
                days_out_to_predict=days_out_to_predict)

        else:
            return render_template('index1.html',
                date_horizon = datetime.today().strftime('%m/%d/%Y'),
                earthquake_horizon = '',
                current_value=0,
                days_out_to_predict=days_out_to_predict)


In [28]:
@app.route("/select", methods=['POST', 'GET'])
def select():
        if request.method == 'POST':
            locations=request.form["locations"]
            earthquake_horizon,date= get_earth_quake_estimates1(locations, pred)
            date=date.split()
            return render_template('index1.html',
                earthquake_horizon = earthquake_horizon,
                date=date,
                days_out_to_predict=days_out_to_predict)

        else:
            return render_template('index1.html',
                date_horizon = datetime.today().strftime('%m/%d/%Y'),
                earthquake_horizon = '',
                current_value=0,
                days_out_to_predict=days_out_to_predict)


In [ ]:
if __name__ == '__main__':
     app.run(debug=False, port=200)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:200
Press CTRL+C to quit
127.0.0.1 - - [28/Mar/2024 15:52:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2024 15:52:42] "POST / HTTP/1.1" 200 -


['Chile-99.67', 'Japan-99.64', 'Philippines-99.73']


127.0.0.1 - - [28/Mar/2024 15:52:58] "POST / HTTP/1.1" 200 -


['Philippines-99.70']
